In [3]:
from Queue import PriorityQueue

#represent the grid with lists
class PuzzleNode:
    def __init__(self, state, fval, gval, parent=None):
        self.parent = parent
        self.state = state
        self.fval = fval # gval + hval
        self.gval = gval # cost to node
        self.pruned = False # not pruned yet
        
    
    def isSolvable(self):
        """ If number of inversions is odd, its unsolvable
        
        The steps are to flatten state, and go through each tile
        and see if the ones following it are less.
        If its not divisible by 2, it is not a solvable state """
        
        inversions = 0
        
        # change from nested lists into a flat, single list
        flat_state = [tile for row in self.state for tile in row]
        
        for i in range(len(flat_state)):
            # see if the tiles following the given tile are 
            # smaller than it (but not zero/blank)
            first_tile = flat_state[i]
            for other_tile in flat_state[i:]:            
                if other_tile < first_tile and other_tile > 0:
                    inversions +=1
        
        # return True if inversions are even, False if inversions are odd            
        return inversions % 2 == 0
    
    # print the state, which is n rows with n values each
    def __str__(self):
        return "\n".join([" ".join([str(x) for x in row]) 
                          for row in self.state])
    
    # Compare fvals to order in queue by cost
    def __lt__(self, other):
        return self.fval < other.fval
    
    # finding where the blank tile is
    def blank_pos(self):
        
        #find which row 0 is in
        for n, row in enumerate(self.state):
            if 0 in row:
                y = n
                break
                
        #use the above to find index of 0 in the row
        x = self.state[y].index(0)
        return y, x # y is first because it is the row
      
def misplaced(state):
    
    #create a single list from the nested lists
    flat_state = [tile for row in state for tile in row]
    misplaced = 0
    correct = 0 #counter
    
    #go through each tile and compare with the correct tile in that position
    for tile in flat_state:
        if tile != correct:
            misplaced += 1
        correct +=1
        
    return misplaced


def manhattan(state):
    
    n = len(state)
    
    # made state into single list from nested lists
    flat_state = [tile for row in state for tile in row]
    count = 0
    
    """ Referenced https://stackoverflow.com/questions/16318757/
    (cont. link) calculating-manhattan-distance-in-python-in-an-8-puzzle-game
    Manhattan distance is the sum of the distances between 
    (actual x coord and tile x coord) and (actual y coord and tile y coord) """
    
    for i, item in enumerate(flat_state):
        prev_row, prev_col = int(i/ n) , i % n
        goal_row, goal_col = int(item /n),item % n
        count += abs(prev_row - goal_row) + abs(prev_col - goal_col)

    return count

In [4]:
def solvepuzzle(n, state, cur_heur, prnt=False):
    
    """Check the format of the state
    it should have n rows, contain all of the values from 0 to n**2-1 only once,
    and have n elements in each row """
    
    flat = [tile for row in state for tile in row]
    sorted_state = sorted(flat)
    if len(state) != n or any(len(state[i]) != n for i in range(n)) or sorted_state != range(n**2):
        # if it's not correctly formatted, return error code -1
        return 0, 0, -1
    
    #create the first node
    start_node = PuzzleNode(state, cur_heur(state), 0)
    
    #if the state is not a solvable state, return error code -2
    if not start_node.isSolvable:
        return 0,0,-2
    
    ### Adapted from code from class session 3.1
    
    # Define goal
    #Via creating 3 rows, and for each row, create n values,
    # adding the value of the column to it
    goal = [[row * n + column for column in range(n)] for row in range(n)]
    
    # Dictionary with current cost to reach all visited nodes
    # use str because list is unhashable
    costs_db = {str(state): start_node}
    
    # Frontier stored as priority queue to maintain ordering
    frontier = PriorityQueue()
    frontier.put(start_node)
    
    # next moves are either move up, down, left or right
    possible_moves = [(-1, 0), (1,0), (0,-1), (0,1)]

    # Begin A* Search
    expansion_counter = 0

    while not frontier.empty():
        # Take the next available node from the priority queue
        cur_node = frontier.get()
    
        if cur_node.pruned:
            continue # Skip if this node has been marked for removal
    
        # Check if we are at the goal
        if cur_node.state == goal: 
            break
        
        # get the coordinates for the blank tile (where 0 is)
        blank_row, blank_col = cur_node.blank_pos()

        # Expand node
        for add_col, add_row in possible_moves:
            
            # only consider moves that are within the board
            # i.e. switching tiles down when you are already at 
            # the bottom row is not an option
            col_move = blank_col + add_col
            row_move = blank_row + add_row
            
            if col_move in range(0,n) and row_move in range(0,n):
                # generate a child state having done the move
                child_state = [row[:] for row in cur_node.state]
                child_state[row_move][col_move] = 0 
                
                # switch blank tile and the value that was in 
                # the spot the blank tile moved to
                value_to_switch_w_blank = cur_node.state[row_move][col_move]
                child_state[blank_row][blank_col] = value_to_switch_w_blank
                
                # cost goes up by 1 since we just added a child node
                gval = cur_node.gval + 1
                
                # If the child node is already in the cost database (i.e. explored)
                if str(child_state) in costs_db:
                    if costs_db[str(child_state)].gval > gval:
                        # Mark existing value for deletion from frontier
                        costs_db[str(child_state)].pruned = True 
                    else:
                        # ignore this child, 
                        # since a better path has already been found previously.
                        continue 
    
                hval = cur_heur(child_state) # Heuristic cost from next node to goal
        
                # Create new node for child
                next_node = PuzzleNode(child_state, gval+hval, gval, cur_node) 
                frontier.put(next_node)
                costs_db[str(child_state)] = next_node #Mark the node as explored
    
        expansion_counter += 1
        
    if prnt:
        print "Frontier size: ", frontier.qsize()
        print "Number of expansions: ", expansion_counter
        print "Error code: ", 0
        
        states = [cur_node.state]
        while cur_node.parent is True:
            states.append((cur_node.parent).state)
            cur_node = cur_node.parent
        
        # print the states
        for state in states:
            print state
        
    return frontier.qsize(), expansion_counter, 0

In [2]:
# TEST
states = [[[5,7,6],[2,4,3],[8,1,0]], 
          [[7,0,8],[4,6,1],[5,3,2]], 
          [[2,3,7],[1,8,0],[6,5,4]]]
heuristics = [misplaced, manhattan]

for board in states:
    print board
    for heuristic in heuristics:
        print solvepuzzle(len(board), board, heuristic, False)


[[5, 7, 6], [2, 4, 3], [8, 1, 0]]
(21099, 65220, 0)
(2394, 4146, 0)
[[7, 0, 8], [4, 6, 1], [5, 3, 2]]
(12408, 28099, 0)
(1852, 3207, 0)
[[2, 3, 7], [1, 8, 0], [6, 5, 4]]
(416, 679, 0)
(89, 148, 0)
